# Fundamentos de `Structured Streaming`

In [ ]:
import org.apache.spark._

In [ ]:
import org.apache.spark.streaming._

In [ ]:
import org.apache.spark.sql.streaming._

In [ ]:
import org.apache.spark.sql.SparkSession

In [ ]:
val spark = SparkSession.builder.getOrCreate()

In [ ]:
import spark.implicits._

In [ ]:
val datos_no_format = spark.readStream.format("socket").
option("host","localhost").option("port", 19001).load()

# Salida de datos

In [ ]:
val query = datos_no_format.writeStream.format("console").
outputMode("append").queryName("test").
start()

# Ciclo de vida del procesamiento

In [ ]:
spark.streams.active

In [ ]:
query.id

In [ ]:
query.runId

In [ ]:
query.name

In [ ]:
query.lastProgress

In [ ]:
query.recentProgress

In [ ]:
query.stop

# Tratamiento de los datos

In [ ]:
import org.apache.spark.sql.types._

In [ ]:
import org.apache.spark.sql.Row

In [ ]:
import org.apache.spark.sql.functions._

In [ ]:
val datos_no_format = spark.readStream.format("socket").
option("host","localhost").option("port", 19001).load()

In [ ]:
datos_no_format.printSchema

In [ ]:
val datos_splitted = datos_no_format.withColumn("tmp", split($"value", ";"))

In [ ]:
datos_splitted.printSchema

In [ ]:
import java.sql.Timestamp

In [ ]:
val datos = datos_splitted.select(from_unixtime($"tmp".getItem(0).cast(LongType) / 1000).cast(TimestampType).as("ts"), 
                                  $"tmp".getItem(1).as("nodo"),
                                  $"tmp".getItem(2).as("sensor"),
                                  $"tmp".getItem(3).as("valor"))

In [ ]:
datos.printSchema

In [ ]:
val query_1 = datos.writeStream.format("console").
outputMode("append").queryName("test 1").
start()

In [ ]:
val query_2 = datos.filter('sensor === "Temp" && 'valor > 26.25).
writeStream.format("console").outputMode("append").queryName("test 2").
start()

In [ ]:
val query_3 = datos.filter('sensor === "Temp" && 'valor > 26.25).
writeStream.format("console").outputMode("update").queryName("test 3").
start()

In [ ]:
query_1.stop

In [ ]:
query_2.stop

In [ ]:
query_3.stop

In [ ]:
val query_4 = datos.writeStream.format("console").
outputMode("append").queryName("test 4").
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
val query_5 = datos.filter('sensor === "Temp" && 'valor > 26.25).
writeStream.format("console").outputMode("append").queryName("test 5").
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
query_4.stop

In [ ]:
query_5.stop

# Ventanas temporales de procesamiento

In [ ]:
val aux_query_5 = datos.groupBy($"sensor", window($"ts", "45 seconds", "15 seconds")).count()

In [ ]:
val query_5 = aux_query_5.writeStream.format("console").outputMode("complete").queryName("test 5").
option("truncate", false).
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
query_5.stop

In [ ]:
val query_6 = datos.writeStream.format("console").
outputMode("append").queryName("test 6").
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
query_6.stop